![title](titulo.png)

**INSTITUTO NACIONAL DE PESQUISAS ESPACIAIS** 

Disciplina: Introdution to Data Science
    
Professores: Rafael Santos e Gilberto Queiroz
    
Acadêmica: Marcelly Homem Coelho
    
Contato: marcellyhc@gmail.com 

**Título:** Aplicação de Técnicas de Data Science no Desenvolvimento de um Sistema para Manutenção Aeronáutica Baseada em Condição 

**Descrição:** Este programa tem como objetivo analisar as mensagens de falha e as remoções dos sistemas das aeronaves.

In [1]:
# Importar as bibliotecas

import numpy as np
import pandas as pd
import seaborn as sns

import random

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
%matplotlib inline

# 1. Investigação Inicial da Estrutura e Conteúdo do Arquivo de Falha

In [2]:
# Criar um dataframe para entrada de dados de falha (arquivo do tipo .csv)

df_dataFailure = pd.read_csv('Data_Failure_Code.csv')   

In [3]:
# Exibir o cabeçalho do dataframe

df_dataFailure.head()

,Aircraft;Flight Phase;Date;Fault Text;Report Maintenance
0,2640;;14/05/2006 16:19;FDE_Outhers02;
1,2640;Enroute Cruise;01/07/2006 15:17;FDE_B_Sys...
2,2640;Enroute Cruise;01/07/2006 15:17;FDE_C_Sys...
3,2640;Enroute Cruise;02/07/2006 04:48;FDE_B_Sys...
4,2640;Enroute Cruise;02/07/2006 04:48;FDE_C_Sys...


In [4]:
# Separar o dataframe por colunas 

df_dataFailure = df_dataFailure['Aircraft;Flight Phase;Date;Fault Text;Report Maintenance'].str.split(';', expand=True)

In [5]:
# Exibir o cabeçalho do dataframe

df_dataFailure.head()

,0,1,2,3,4
0,2640,,14/05/2006 16:19,FDE_Outhers02,
1,2640,Enroute Cruise,01/07/2006 15:17,FDE_B_System3,RMA_A_System3
2,2640,Enroute Cruise,01/07/2006 15:17,FDE_C_System3,RMA_A_System3
3,2640,Enroute Cruise,02/07/2006 04:48,FDE_B_System3,RMA_A_System3
4,2640,Enroute Cruise,02/07/2006 04:48,FDE_C_System3,RMA_A_System3


In [6]:
# Renomear as colunas do dataframe

df_dataFailure.columns = ['Aircraft', 'Flight Phase', 'Date', 'Fault Text', 'Report Maintenance']  

In [7]:
# Exibir o cabeçalho do dataframe

df_dataFailure.head()

,Aircraft,Flight Phase,Date,Fault Text,Report Maintenance
0,2640,,14/05/2006 16:19,FDE_Outhers02,
1,2640,Enroute Cruise,01/07/2006 15:17,FDE_B_System3,RMA_A_System3
2,2640,Enroute Cruise,01/07/2006 15:17,FDE_C_System3,RMA_A_System3
3,2640,Enroute Cruise,02/07/2006 04:48,FDE_B_System3,RMA_A_System3
4,2640,Enroute Cruise,02/07/2006 04:48,FDE_C_System3,RMA_A_System3


In [8]:
# Verifica a dimensão do dataframe (qtd linhas, qtd colunas)

df_dataFailure.shape

(7238, 5)

In [9]:
# Verificar o tipo de dado de cada coluna do dataframe

df_dataFailure.dtypes

Aircraft              object
Flight Phase          object
Date                  object
Fault Text            object
Report Maintenance    object
dtype: object

In [10]:
# Converter os dados da coluna 'Aircraft' para o formato inteiro

df_dataFailure['Aircraft'] = df_dataFailure['Aircraft'].astype(int)

In [11]:
# Converter os dados da coluna 'Date' para o formato de data-hora

df_dataFailure['Date'] =  pd.to_datetime(df_dataFailure['Date'], format='%d/%m/%Y %H:%M')

In [12]:
# Verificar o tipo de dado de cada coluna do dataframe

df_dataFailure.dtypes

Aircraft                       int32
Flight Phase                  object
Date                  datetime64[ns]
Fault Text                    object
Report Maintenance            object
dtype: object

In [13]:
# Determinar quantas 'Flight Phase' diferentes há no dataframe

len(df_dataFailure['Aircraft'].unique())

15

In [14]:
# Verificar quais 'Aircraft' apresentaram maior quantidade de Mensagens de Falha  

df_dataFailure['Aircraft'].value_counts()

2766    1059
1950     728
1990     668
1151     626
2640     560
326      475
131      465
2436     421
1419     419
791      417
2838     369
2209     324
312      268
1710     243
2982     196
Name: Aircraft, dtype: int64

$\color{red}{\text{OBSERVAÇÃO:}}$ O Aircraft igual a 2766 é o que apresenta a maior quantidade de Fault Text. Além disso, é o segundo em relação a quantidade de remoções de peças.

## 1.1 Análise das Mensagens de Falha de um Aircraft Específico

In [15]:
# Definir uma variável para a seleção de um arcraft específico

var_aircraftSelected = 2766

In [16]:
# Criar um dataframe para o aircraft específico 

df_dataFailure_airSelec = df_dataFailure[df_dataFailure['Aircraft'] == var_aircraftSelected]

In [17]:
# Exibir o cabeçalho do dataframe

df_dataFailure_airSelec.head()

,Aircraft,Flight Phase,Date,Fault Text,Report Maintenance
1492,2766,Power On,2006-04-19 13:29:00,FDE_Outhers00,RMA_Others04
1493,2766,Power On,2006-04-19 13:29:00,FDE_Outhers00,RMA_Others04
1494,2766,Power On,2006-04-21 16:12:00,FDE_Outhers00,RMA_Others04
1495,2766,Power On,2006-04-21 16:12:00,FDE_Outhers00,RMA_Others04
1496,2766,Initial Climb,2006-04-22 19:59:00,FDE_E_System1,RMA_F_System1


# 2. Investigação Inicial da Estrutura e Conteúdo do Arquivo de Remoção

In [18]:
# Criar um dataframe para entrada de dados de remoção (arquivo do tipo .csv)

df_dataRemoval = pd.read_csv('Data_Removal_Code.csv')  

In [19]:
# Exibir do cabeçalho do dataframe

df_dataRemoval.head()

,Aircraft;Component;System;Date;Reason;Time Hours; Time Cycles
0,1140;MME_Component_A;System1;29/05/2006;3;1181...
1,1140;MME_Component_A;System1;29/05/2006;3;1181...
2,1140;MME_Component_B;System1;29/05/2006;3;1092...
3,1140;MME_Component_B;System3;24/06/2006;3;312;37
4,1140;MME_Component_B;System3;10/07/2006;3;1186...


In [20]:
# Separar o dataframe por colunas 

df_dataRemoval = df_dataRemoval['Aircraft;Component;System;Date;Reason;Time Hours; Time Cycles'].str.split(';', expand=True)

In [21]:
# Exibir o cabeçalho do dataframe

df_dataRemoval.head()

,0,1,2,3,4,5,6
0,1140,MME_Component_A,System1,29/05/2006,3,118123,15961
1,1140,MME_Component_A,System1,29/05/2006,3,118123,15961
2,1140,MME_Component_B,System1,29/05/2006,3,1092,139
3,1140,MME_Component_B,System3,24/06/2006,3,312,37
4,1140,MME_Component_B,System3,10/07/2006,3,118698,16028


In [22]:
# Renomear as colunas do dataframe

df_dataRemoval.columns = ['Aircraft', 'Component', 'System', 'Date', 'Reason', 'Time Hours', 'Time Cycles']  

In [23]:
# Exibir o cabeçalho do dataframe

df_dataRemoval.head()

,Aircraft,Component,System,Date,Reason,Time Hours,Time Cycles
0,1140,MME_Component_A,System1,29/05/2006,3,118123,15961
1,1140,MME_Component_A,System1,29/05/2006,3,118123,15961
2,1140,MME_Component_B,System1,29/05/2006,3,1092,139
3,1140,MME_Component_B,System3,24/06/2006,3,312,37
4,1140,MME_Component_B,System3,10/07/2006,3,118698,16028


In [24]:
# Verifica a dimensão do dataframe (qtd linhas, qtd colunas)

df_dataRemoval.shape

(1284, 7)

In [25]:
# Verificar o tipo de dado de cada coluna do dataframe

df_dataRemoval.dtypes

Aircraft       object
Component      object
System         object
Date           object
Reason         object
Time Hours     object
Time Cycles    object
dtype: object

In [26]:
# Converter os dados da coluna 'Aircraft' e 'Reason' para o formato inteiro

df_dataRemoval['Aircraft'] = df_dataRemoval['Aircraft'].astype(int)
df_dataRemoval['Reason']   = df_dataRemoval['Reason'].astype(int)

In [27]:
# Converter os dados das colunas 'Time Hours' e 'Time Cycles' para o formato float

df_dataRemoval['Time Hours']  = df_dataRemoval['Time Hours'].astype(float)
df_dataRemoval['Time Cycles'] = df_dataRemoval['Time Cycles'].astype(float)

In [28]:
# Converter os dados da coluna 'Date' para o formato de data

df_dataRemoval['Date'] =  pd.to_datetime(df_dataRemoval['Date'], format='%d/%m/%Y')

In [29]:
# Verificar o tipo de dado de cada coluna do dataframe

df_dataRemoval.dtypes

Aircraft                int32
Component              object
System                 object
Date           datetime64[ns]
Reason                  int32
Time Hours            float64
Time Cycles           float64
dtype: object

In [30]:
# Determinar quantos 'Component' diferentes há no dataframe

len(df_dataRemoval['Component'].unique())

17

In [31]:
# Verificar quais 'Component' foram mais trocados 

df_dataRemoval['Component'].value_counts()

MME_Component_B    270
MME_Component_A    210
MME_Component_D    177
MME_Component_F    113
MME_Component_J     87
MME_Component_G     86
MME_Component_H     77
MME_Component_I     64
MME_Component_N     45
MME_Component_E     38
MME_Component_L     37
MME_Component_K     25
MME_Component_O     22
MME_Component_M     19
MME_Component_C      6
MME_Component_P      5
MME_Component_Q      3
Name: Component, dtype: int64

In [32]:
# Verificar quais 'Aircraft' realizaram mais trocas de componentes  

df_dataRemoval['Aircraft'].value_counts()

2640    99
2766    94
2361    93
2326    91
2567    86
1950    78
2982    74
1399    62
2838    60
2436    59
131     55
1151    54
312     53
1990    50
1419    49
736     46
1710    41
2209    39
791     37
326     30
1140    25
165      9
Name: Aircraft, dtype: int64

## 2.1 Análise das Remoções de um Aircraft Específico

In [33]:
# Criar um dataframe para o aircraft específico 

df_dataRemoval_airSelec = df_dataRemoval[df_dataRemoval['Aircraft'] == var_aircraftSelected]

In [34]:
# Exibir o cabeçalho do dataframe

df_dataRemoval.head()

,Aircraft,Component,System,Date,Reason,Time Hours,Time Cycles
0,1140,MME_Component_A,System1,2006-05-29,3,118123.0,15961.0
1,1140,MME_Component_A,System1,2006-05-29,3,118123.0,15961.0
2,1140,MME_Component_B,System1,2006-05-29,3,1092.0,139.0
3,1140,MME_Component_B,System3,2006-06-24,3,312.0,37.0
4,1140,MME_Component_B,System3,2006-07-10,3,118698.0,16028.0
